In [1]:
import pandas as pd
df=pd.read_csv("D:/face attendence data.csv")
date=input('Enter date-dd/mm/yyyy')
df[date]='A'
df.to_csv("D:/face attendence data.csv",index=False)
df

Enter date-dd/mm/yyyy21/05/2024


,S.no,Unnamed: 1,Unnamed: 2,21/05/2024


In [2]:
import pandas as pd
import cv2

face_cascade = cv2.CascadeClassifier("D:/haarcascade_frontalface_default.xml")
cam = cv2.VideoCapture(0)

Id = input('Enter your id:')
Name = input('Enter your name:')

df = pd.read_csv("D:/face attendence data.csv")
df2 = pd.DataFrame({'Id': [Id], 'Name': [Name]})
df = pd.concat([df, df2]).drop_duplicates().reset_index(drop=True)
df.to_csv("D:/face attendence data.csv", index=False)
sampleNum = 0
while True:
    ret, img = cam.read()
    if not ret:
        break
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (144, 1, 1), 3)
        sampleNum += 1
        # Save the captured face in the dataset folder
        file_path = f"D:/Face/image.{Id}-{sampleNum}.jpg"
        cv2.imwrite(file_path, img[y:y+h, x:x+w])
        cv2.imshow('frame', img)
    if cv2.waitKey(100) & 0xff == ord('q'):
        break
    elif sampleNum >= 10:
        break

# Release the video capture and close windows
cam.release()
cv2.destroyAllWindows()


Enter your id:88
Enter your name:deepak


In [3]:
import os
import cv2
import numpy as np
from PIL import Image

recognizer = cv2.face.LBPHFaceRecognizer_create()
path = "D:/Face"

def getImagewithID(path):
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    faces = []
    IDs = []
    for imagePath in imagePaths:
        # Open image and convert to grayscale
        faceImg = Image.open(imagePath).convert('L')
        faceNp = np.array(faceImg, 'uint8')
        # Extract the ID from the filename assuming the format is image.ID-SAMPLENUMBER.jpg
        try:
            ID = int(os.path.split(imagePath)[-1].split('.')[1].split('-')[0])
            faces.append(faceNp)
            IDs.append(ID)
            cv2.imshow('training', faceNp)
            cv2.waitKey(100)
        except IndexError:
            print(f"Skipping file {imagePath}, filename format is incorrect")
    return np.array(IDs), faces

ids, faces = getImagewithID(path)
recognizer.train(faces, ids)
recognizer.save("D:/trainningData.yml")
cv2.destroyAllWindows()


In [4]:
import cv2
import numpy as np
import pandas as pd

date = input('Enter date-dd/mm/yyyy')
faceDetect = cv2.CascadeClassifier("D:/haarcascade_frontalface_default.xml")
cam = cv2.VideoCapture(0)
rec = cv2.face.LBPHFaceRecognizer_create()
rec.read("D:/trainningData.yml")
df = pd.read_csv("D:/face attendence data.csv")
if date not in df.columns:
    df[date] = 'A'

font = cv2.FONT_HERSHEY_COMPLEX_SMALL

while True:
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceDetect.detectMultiScale(gray, 1.3, 5)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        id, conf = rec.predict(gray[y:y + h, x:x + w])
        
        if conf < 100:  # You can adjust the confidence threshold as needed
            name = df['Name'][df['Id'] == id].values[0]
            display_text = f"ID: {id} Name: {name}"
            df.loc[df['Id'] == id, date] = 'P'
        else:
            display_text = "Unknown"
        
        cv2.putText(img, display_text, (x, y + h + 20), font, 1, (255, 255, 255), 2)
    
    df.to_csv("D:/face attendence data.csv", index=False)
    cv2.imshow('Face', img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()


Enter date-dd/mm/yyyy21/05/2024
